# Bayesian linear regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import subprocess

In [ ]:
# Initialize true parameters
dim = 2
centers = [np.array([-5, 0]), np.array([5, 0]), np.array([0, 5])]

In [ ]:
# Utility to save files with Unix-like newlines
def save_np(filename, npobj):
    with open(filename, 'wb') as f:
        np.savetxt(f, npobj, fmt='%1.5f')

In [ ]:
# Generate data
rng = 20201124
np.random.seed(rng)
n = 200
xx = np.zeros((n, dim))
cc = int(n/3)*[0] + int(n/3)*[1] + (n - 2*int(n/3))*[2]
yy = np.zeros(n)
for i in range(n):
    x = np.random.multivariate_normal(mean=centers[cc[i]],
                                      cov=np.identity(dim))
    xx[i,:] = x
    # mu = np.dot(x, centers[cc[i]])
    mu = 4 * cc[i]
    y = np.random.normal(loc=mu, scale=1) # TODO ?
    yy[i] = y
# Save to file
save_np("../resources/csv/in/logsb_cov_mix.csv", xx)
save_np("../resources/csv/in/logsb_data.csv", yy)

In [ ]:
# Generate grid points
xx_grid = np.matrix([0, 5])
yy_grid = np.arange(-1, +9.1, 0.5)
# Save to file
save_np("../resources/csv/in/logsb_grid_cov_mix.csv", xx_grid)
save_np("../resources/csv/in/logsb_grid_data.csv", yy_grid)

In [ ]:
# Run the executable
cmd = ('../build/run ../algo_settings.asciipb '
    'NNIG ../resources/asciipb/nnig_ngg_prior.asciipb '
    'LogSB ../resources/asciipb/lsb_normal_prior.asciipb "" '
    '../resources/csv/in/logsb_data.csv ../resources/csv/in/logsb_grid_data.csv '
    '../resources/csv/out/logsb_dens.csv ../resources/csv/out/logsb_nclu.csv '
    '../resources/csv/out/logsb_clus.csv "" "" '
    '../resources/csv/in/logsb_cov_mix.csv 
    '../resources/csv/in/logsb_grid_cov_mix.csv').split()
subprocess.run(cmd, capture_output=True)

## Plot density

In [ ]:
matr = np.genfromtxt("../resources/csv/out/logsb_dens.csv", delimiter=',')

In [ ]:
plt.plot(yy_grid, np.exp(matr[5,:]))
plt.plot(yy_grid, np.exp(matr[29,:]))
plt.plot(yy_grid, np.exp(matr[400,:]))
plt.plot(yy_grid, np.exp(np.mean(matr, axis=0)), linestyle='--')
plt.title("Densities")